In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
# from sklearn.impute import SimpleImputer

from utils.preprocessing import OneHotEncoderFeatures_, GenericNumericEstimator_, SimpleImputer_

In [2]:
# Carregando base com target e features
df = pd.read_csv('features_and_target_df.csv')
df_ids = df[['IdAluno','IdTurma','IdPeriodo','IdSerie','AnoMesRefFeatures','AnoRefFeatures','DataRefFeaturesUltDia']]
X = df.drop(columns=['TargetDesistente','IdAluno','IdTurma','IdPeriodo','IdSerie','AnoMesRefFeatures','AnoRefFeatures','DataRefFeaturesUltDia'])
y = df['TargetDesistente']

In [3]:
X.shape

(6955, 83)

In [4]:
# Separando as bases de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=3566)

In [5]:
X = X.loc[y.index]


In [6]:
# Separando as features entre numericas e categoricas
lst_features_categories = ['TipoTurma', 'TurnoPrincipal', 'IdDisciplina',
       'DisciplinaRequerNota', 
       'DisciplinaStUsaNotaConceito', 'SexoTurma', 'IdProfessor', 'Sexo',
       'EstadoCivil', 'CorRaca', 'StDeficienciaCegueira',
       'StDeficienciaBaixaVisao', 'StDeficienciaSurdez',
       'StDeficienciaAuditiva', 'StDeficienciaFisica',
       'StDeficienciaSurdoCegueira', 'StDeficienciaMultipla',
       'StDeficienciaMental', 'StDeficienciaAutismoInfantil',
       'StDeficienciaSindromeAsperger', 'StDeficienciaSindromeRett',
       'StDeficienciaTrastornoDesintegrativo', 'StDeficienciaAltasHabilidades',
       'StAppComunicacao_Sincronizado', 'StRecursoAuxilioLedor',
       'StRecursoAuxilioTranscricao', 'StRecursoGuiaInterprete',
       'StRecursoTradutorInterpreteDeLibras', 'StRecursoLeituraLabial',
       'StRecursoMaterialDidaticoProvaBraille', 'StRecursoProvaAmpliada',
       'StRecursoProvaSuperampliada', 'StRecursoCdComAudio',
       'StRecursoLinguaPortuguesaSegundaLingua', 'StRecursoProvaEmVideo',
       'StPermiteUsoImagem', 'StEstrangeiro', 'TipoResponsavel',
       'GrauParentescoResponsavel']

lst_features_numerics = [col for col in X.columns if col not in lst_features_categories]

In [7]:
X_train.shape

(4868, 83)

In [8]:
X_test.shape

(2087, 83)

In [9]:
# Lista de modelos de classficação que não sofrem a a ecistência de Nulls 
models = {
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier(),
    'Naive Bayes': GaussianNB(),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    # 'Gradient Boosting': GradientBoostingClassifier(),
    'XGBoost': XGBClassifier()
}


# Evaluate each model
results = {}
for model_name, model in models.items():
    # Aplicando o pipeline de preprocessamento e o modelo
    pipe = Pipeline(
        [
            (
                'Scaling', 
                GenericNumericEstimator_(
                    estimator=MinMaxScaler(), 
                    type_numeric=['floating','integer'], 
                    features_to_ignore=lst_features_categories
                )
            ),
            ('Fill Nulls', SimpleImputer_(ls_features=lst_features_numerics)),
            (
                'OneHotEncoderFeatures', 
                OneHotEncoderFeatures_(
                    ls_features=lst_features_categories, 
                    nulls_to_base=False, 
                    handle_unknown='ignore',
                    target='TargetDesistente'
                )
            ),
            ('Model', model)
        ]
    )
    
    # Fit do modelo
    pipe.fit(X_train, y_train)
    
    # Gerando previsões com o modelo ajustado
    y_pred = pipe.predict(X_test)
    y_proba = pipe.predict_proba(X_test)[:, 1]
    
    # Calculando métricas para a avaliação do modelo
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc_roc = roc_auc_score(y_test, y_proba)
    
    # Guardando as métricas calculadas para comparar o desempenho dos modelos testados
    results[model_name] = {
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1-Score': f1,
        'AUC-ROC': auc_roc
    }
    
    # Print classification report
    print(f"Model: {model_name}\n")
    print(classification_report(y_test, y_pred))
    print("="*60)

# Salvando os resultados
results_df = pd.DataFrame(results).T
results_df.reset_index().rename(columns={'index':'Model'}, inplace=True)
print(results_df)

# Salvando os resultados em um arquivo
results_df.to_csv('comparacao_entre_modelos.csv')


c:\Users\johnnathan.lopes\Documents\projetos\pos-tech-datathon\utils\preprocessing.py:78: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  value_positive = float(df[target].value_counts(normalize=True).to_frame().reset_index().query("index == 1")[target]) # proporção de sucesso
c:\Users\johnnathan.lopes\Documents\projetos\pos-tech-datathon\utils\preprocessing.py:78: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  value_positive = float(df[target].value_counts(normalize=True).to_frame().reset_index().query("index == 1")[target]) # proporção de sucesso
c:\Users\johnnathan.lopes\Documents\projetos\pos-tech-datathon\utils\preprocessing.py:78: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  value_positive 

Model: Decision Tree

              precision    recall  f1-score   support

           0       0.91      0.89      0.90      1719
           1       0.55      0.61      0.58       368

    accuracy                           0.84      2087
   macro avg       0.73      0.75      0.74      2087
weighted avg       0.85      0.84      0.85      2087



c:\Users\johnnathan.lopes\Documents\projetos\pos-tech-datathon\utils\preprocessing.py:78: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  value_positive = float(df[target].value_counts(normalize=True).to_frame().reset_index().query("index == 1")[target]) # proporção de sucesso
c:\Users\johnnathan.lopes\Documents\projetos\pos-tech-datathon\utils\preprocessing.py:78: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  value_positive = float(df[target].value_counts(normalize=True).to_frame().reset_index().query("index == 1")[target]) # proporção de sucesso
c:\Users\johnnathan.lopes\Documents\projetos\pos-tech-datathon\utils\preprocessing.py:78: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  value_positive 

Model: Random Forest

              precision    recall  f1-score   support

           0       0.89      0.97      0.93      1719
           1       0.77      0.47      0.58       368

    accuracy                           0.88      2087
   macro avg       0.83      0.72      0.76      2087
weighted avg       0.87      0.88      0.87      2087

Model: Naive Bayes

              precision    recall  f1-score   support

           0       0.83      0.99      0.91      1719
           1       0.74      0.08      0.14       368

    accuracy                           0.83      2087
   macro avg       0.79      0.54      0.52      2087
weighted avg       0.82      0.83      0.77      2087



c:\Users\johnnathan.lopes\AppData\Local\anaconda3\envs\analysis_conda_env\Lib\site-packages\sklearn\preprocessing\_encoders.py:202: UserWarning: Found unknown categories in columns [37] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\johnnathan.lopes\AppData\Local\anaconda3\envs\analysis_conda_env\Lib\site-packages\sklearn\preprocessing\_encoders.py:202: UserWarning: Found unknown categories in columns [37] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\johnnathan.lopes\Documents\projetos\pos-tech-datathon\utils\preprocessing.py:78: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  value_positive = float(df[target].value_counts(normalize=True).to_frame().reset_index().query("index == 1")[target]) # proporção de sucesso
c:\Users\johnnathan.lopes\Documents\projetos\pos-tech-datathon\utils\prepro

Model: K-Nearest Neighbors

              precision    recall  f1-score   support

           0       0.85      0.96      0.90      1719
           1       0.57      0.23      0.32       368

    accuracy                           0.83      2087
   macro avg       0.71      0.59      0.61      2087
weighted avg       0.80      0.83      0.80      2087



c:\Users\johnnathan.lopes\Documents\projetos\pos-tech-datathon\utils\preprocessing.py:78: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  value_positive = float(df[target].value_counts(normalize=True).to_frame().reset_index().query("index == 1")[target]) # proporção de sucesso
c:\Users\johnnathan.lopes\Documents\projetos\pos-tech-datathon\utils\preprocessing.py:78: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  value_positive = float(df[target].value_counts(normalize=True).to_frame().reset_index().query("index == 1")[target]) # proporção de sucesso
c:\Users\johnnathan.lopes\Documents\projetos\pos-tech-datathon\utils\preprocessing.py:78: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  value_positive 

Model: XGBoost

              precision    recall  f1-score   support

           0       0.91      0.97      0.94      1719
           1       0.80      0.53      0.64       368

    accuracy                           0.89      2087
   macro avg       0.85      0.75      0.79      2087
weighted avg       0.89      0.89      0.88      2087

                     Accuracy  Precision    Recall  F1-Score   AUC-ROC
Decision Tree        0.842837   0.549020  0.608696  0.577320  0.755358
Random Forest        0.881169   0.767857  0.467391  0.581081  0.869031
Naive Bayes          0.832295   0.736842  0.076087  0.137931  0.726708
K-Nearest Neighbors  0.833253   0.568493  0.225543  0.322957  0.700945
XGBoost              0.893148   0.798354  0.527174  0.635025  0.882974


c:\Users\johnnathan.lopes\AppData\Local\anaconda3\envs\analysis_conda_env\Lib\site-packages\sklearn\preprocessing\_encoders.py:202: UserWarning: Found unknown categories in columns [37] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\johnnathan.lopes\AppData\Local\anaconda3\envs\analysis_conda_env\Lib\site-packages\sklearn\preprocessing\_encoders.py:202: UserWarning: Found unknown categories in columns [37] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
